# Neural network prediction
Source:
https://github.com/kristpapadopoulos/seriesnet.git

In [1]:
from __future__ import print_function, division
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers import Conv1D, Input, Add, Activation, Dropout

from keras.models import Sequential, Model

from keras.regularizers import l2

from keras.initializers import TruncatedNormal

from keras.layers.advanced_activations import LeakyReLU, ELU

from keras import optimizers

Using TensorFlow backend.


In [2]:
def DC_CNN_Block(nb_filter, filter_length, dilation, l2_layer_reg):
    def f(input_):
        
        residual =    input_
        
        layer_out =   Conv1D(filters=nb_filter, kernel_size=filter_length, 
                      dilation_rate=dilation, 
                      activation='linear', padding='causal', use_bias=False,
                      kernel_initializer=TruncatedNormal(mean=0.0, stddev=0.05, 
                      seed=42), kernel_regularizer=l2(l2_layer_reg))(input_)
                    
        layer_out =   Activation('selu')(layer_out)
        
        skip_out =    Conv1D(1,1, activation='linear', use_bias=False, 
                      kernel_initializer=TruncatedNormal(mean=0.0, stddev=0.05, 
                      seed=42), kernel_regularizer=l2(l2_layer_reg))(layer_out)
        
        network_in =  Conv1D(1,1, activation='linear', use_bias=False, 
                      kernel_initializer=TruncatedNormal(mean=0.0, stddev=0.05, 
                      seed=42), kernel_regularizer=l2(l2_layer_reg))(layer_out)
                      
        network_out = Add()([residual, network_in])
        
        return network_out, skip_out
    
    return f


In [34]:
def DC_CNN_Model(length):
    
    input = Input(shape=(length,1))
    
    l1a, l1b = DC_CNN_Block(32,2,1,0.001)(input)    
    l2a, l2b = DC_CNN_Block(32,2,2,0.001)(l1a) 
    l3a, l3b = DC_CNN_Block(32,2,4,0.001)(l2a)
    l4a, l4b = DC_CNN_Block(32,2,8,0.001)(l3a)
    l5a, l5b = DC_CNN_Block(32,2,16,0.001)(l4a)
    l6a, l6b = DC_CNN_Block(32,2,32,0.001)(l5a)
    l6b = Dropout(0.8)(l6b) #dropout used to limit influence of earlier data
    l7a, l7b = DC_CNN_Block(32,2,64,0.001)(l6a)
    l7b = Dropout(0.8)(l7b) #dropout used to limit influence of earlier data

    l8 =   Add()([l1b, l2b, l3b, l4b, l5b, l6b, l7b])
    
    l9 =   Activation('relu')(l8)
           
    l21 =  Conv1D(1,1, activation='linear', use_bias=False, 
           kernel_initializer=TruncatedNormal(mean=0.0, stddev=0.05, seed=42),
           kernel_regularizer=l2(0.001))(l9)

    model = Model(inputs=input, outputs=l21)
    
    adam = tf.keras.optimizers.Adam(lr=0.00075, beta_1=0.9, beta_2=0.999, epsilon=None,  decay=0.0, amsgrad=False)
    
    #model.compile(loss='kullback_leibler_divergence', optimizer=adam, metrics=['mse']) #not so good
    model.compile(loss='mae', optimizer=adam, metrics=['mse']) # best
    
    return model

In [35]:
#url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
df = pd.read_csv(url)
df2 = df.loc[(df["Country/Region"] == "Italy"),'1/29/20':]
#df2.loc[:,'3/23/20']= [167];
#df2 = df2.assign(next = [9134]) 
arr = df2.to_numpy()
#arr = list(np.arange(60,df2.shape[1]+60))
#arr = arr[arr != 0]
print(arr.shape[1])
i = 0
b = []
while i < arr.shape[1]:
    temp = arr[0][i]-arr[0][i-1]
    if temp > 0:
        b.append(temp)
    i += 1
timeseries = np.array(b)
timeseries

# timeseries input is 1-D numpy array
# forecast_size is the forecast horizon

timeseries = timeseries[~pd.isna(timeseries)]

length = len(timeseries)-1

timeseries = np.atleast_2d(np.asarray(timeseries))
if timeseries.shape[0] == 1:
    timeseries = timeseries.T 

model = DC_CNN_Model(length)
print('\n\nModel with input size {}, output size {}'.
                            format(model.input_shape, model.output_shape))

model.summary()

X = timeseries[:-1].reshape(1,length,1)
y = timeseries[1:].reshape(1,length,1)

model.fit(X, y, epochs=3000)
   


64


Model with input size (None, 39, 1), output size (None, 39, 1)
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 39, 1)        0                                            
__________________________________________________________________________________________________
conv1d_67 (Conv1D)              (None, 39, 32)       64          input_4[0][0]                    
__________________________________________________________________________________________________
activation_25 (Activation)      (None, 39, 32)       0           conv1d_67[0][0]                  
__________________________________________________________________________________________________
conv1d_69 (Conv1D)              (None, 39, 1)        32          activation_25[0][0]              
________________________

Epoch 1/3000
1/1 [==============================] - 1s 668ms/step - loss: 337.2662 - mse: 218850.9688
Epoch 2/3000
1/1 [==============================] - 0s 7ms/step - loss: 337.2385 - mse: 218808.2500
Epoch 3/3000
1/1 [==============================] - 0s 8ms/step - loss: 337.2246 - mse: 218792.7656
Epoch 4/3000
1/1 [==============================] - 0s 8ms/step - loss: 337.2175 - mse: 218784.9219
Epoch 5/3000
1/1 [==============================] - 0s 8ms/step - loss: 337.2042 - mse: 218767.3281
Epoch 6/3000
1/1 [==============================] - 0s 7ms/step - loss: 337.1283 - mse: 218664.9219
Epoch 7/3000
1/1 [==============================] - 0s 7ms/step - loss: 337.1068 - mse: 218650.4375
Epoch 8/3000
1/1 [==============================] - 0s 7ms/step - loss: 337.0582 - mse: 218569.0000
Epoch 9/3000
1/1 [==============================] - 0s 7ms/step - loss: 337.0384 - mse: 218553.6406
Epoch 10/3000
1/1 [==============================] - 0s 16ms/step - loss: 337.0122 - mse: 218521.0

Epoch 82/3000
1/1 [==============================] - 0s 7ms/step - loss: 272.2185 - mse: 143093.5000
Epoch 83/3000
1/1 [==============================] - 0s 8ms/step - loss: 270.1159 - mse: 141878.7344
Epoch 84/3000
1/1 [==============================] - 0s 7ms/step - loss: 255.8380 - mse: 125958.8594
Epoch 85/3000
1/1 [==============================] - 0s 8ms/step - loss: 250.4673 - mse: 120988.9375
Epoch 86/3000
1/1 [==============================] - 0s 11ms/step - loss: 232.3510 - mse: 103106.3047
Epoch 87/3000
1/1 [==============================] - 0s 7ms/step - loss: 244.2977 - mse: 118819.5859
Epoch 88/3000
1/1 [==============================] - 0s 7ms/step - loss: 244.6117 - mse: 115839.6953
Epoch 89/3000
1/1 [==============================] - 0s 7ms/step - loss: 232.8228 - mse: 110944.2344
Epoch 90/3000
1/1 [==============================] - 0s 8ms/step - loss: 204.8342 - mse: 83786.5391
Epoch 91/3000
1/1 [==============================] - 0s 8ms/step - loss: 216.9418 - mse: 92

Epoch 164/3000
1/1 [==============================] - 0s 39ms/step - loss: 85.8017 - mse: 17473.5410
Epoch 165/3000
1/1 [==============================] - 0s 10ms/step - loss: 73.9011 - mse: 13920.6855
Epoch 166/3000
1/1 [==============================] - 0s 6ms/step - loss: 61.6173 - mse: 10090.2373
Epoch 167/3000
1/1 [==============================] - 0s 7ms/step - loss: 57.3216 - mse: 8512.6533
Epoch 168/3000
1/1 [==============================] - 0s 7ms/step - loss: 68.8838 - mse: 12034.5088
Epoch 169/3000
1/1 [==============================] - 0s 8ms/step - loss: 65.1866 - mse: 10587.9521
Epoch 170/3000
1/1 [==============================] - 0s 7ms/step - loss: 86.1373 - mse: 16340.2148
Epoch 171/3000
1/1 [==============================] - 0s 40ms/step - loss: 61.0445 - mse: 8581.1846
Epoch 172/3000
1/1 [==============================] - 0s 9ms/step - loss: 75.5763 - mse: 12329.7773
Epoch 173/3000
1/1 [==============================] - 0s 8ms/step - loss: 63.1638 - mse: 10073.2656

Epoch 247/3000
1/1 [==============================] - 0s 8ms/step - loss: 59.1463 - mse: 8811.6875
Epoch 248/3000
1/1 [==============================] - 0s 8ms/step - loss: 57.8367 - mse: 8851.2207
Epoch 249/3000
1/1 [==============================] - 0s 7ms/step - loss: 57.0040 - mse: 8672.0469
Epoch 250/3000
1/1 [==============================] - 0s 7ms/step - loss: 56.9797 - mse: 8706.2959
Epoch 251/3000
1/1 [==============================] - 0s 7ms/step - loss: 58.2085 - mse: 9042.7861
Epoch 252/3000
1/1 [==============================] - 0s 13ms/step - loss: 57.3531 - mse: 8827.6055
Epoch 253/3000
1/1 [==============================] - 0s 7ms/step - loss: 56.5758 - mse: 8440.2422
Epoch 254/3000
1/1 [==============================] - 0s 7ms/step - loss: 57.1749 - mse: 8720.8994
Epoch 255/3000
1/1 [==============================] - 0s 8ms/step - loss: 55.4912 - mse: 8669.4512
Epoch 256/3000
1/1 [==============================] - 0s 8ms/step - loss: 57.5727 - mse: 8919.2148
Epoch 257

...

Epoch 2634/3000
1/1 [==============================] - 0s 5ms/step - loss: 31.9655 - mse: 3954.3137
Epoch 2635/3000
1/1 [==============================] - 0s 6ms/step - loss: 32.1849 - mse: 3912.8730
Epoch 2636/3000
1/1 [==============================] - 0s 7ms/step - loss: 31.4403 - mse: 3848.5632
Epoch 2637/3000
1/1 [==============================] - 0s 7ms/step - loss: 31.6341 - mse: 3814.4412
Epoch 2638/3000
1/1 [==============================] - 0s 7ms/step - loss: 31.8965 - mse: 3836.5332
Epoch 2639/3000
1/1 [==============================] - 0s 38ms/step - loss: 32.4112 - mse: 3888.6462
Epoch 2640/3000
1/1 [==============================] - 0s 9ms/step - loss: 31.2602 - mse: 3813.6724
Epoch 2641/3000
1/1 [==============================] - 0s 7ms/step - loss: 31.7767 - mse: 3898.2874
Epoch 2642/3000
1/1 [==============================] - 0s 8ms/step - loss: 31.7201 - mse: 3868.4463
Epoch 2643/3000
1/1 [==============================] - 0s 7ms/step - loss: 31.4319 - mse: 3785.4412

Epoch 2716/3000
1/1 [==============================] - 0s 7ms/step - loss: 31.2781 - mse: 3738.9094
Epoch 2717/3000
1/1 [==============================] - 0s 6ms/step - loss: 31.5111 - mse: 3795.7507
Epoch 2718/3000
1/1 [==============================] - 0s 7ms/step - loss: 31.3799 - mse: 3814.6843
Epoch 2719/3000
1/1 [==============================] - 0s 7ms/step - loss: 31.3775 - mse: 3828.8701
Epoch 2720/3000
1/1 [==============================] - 0s 7ms/step - loss: 31.4002 - mse: 3829.7961
Epoch 2721/3000
1/1 [==============================] - 0s 42ms/step - loss: 31.7893 - mse: 3976.1545
Epoch 2722/3000
1/1 [==============================] - 0s 10ms/step - loss: 31.2194 - mse: 3859.0986
Epoch 2723/3000
1/1 [==============================] - 0s 8ms/step - loss: 31.2302 - mse: 3810.7207
Epoch 2724/3000
1/1 [==============================] - 0s 7ms/step - loss: 31.6284 - mse: 3791.3711
Epoch 2725/3000
1/1 [==============================] - 0s 8ms/step - loss: 31.2104 - mse: 3758.817

Epoch 2798/3000
1/1 [==============================] - 0s 7ms/step - loss: 32.0676 - mse: 3788.6414
Epoch 2799/3000
1/1 [==============================] - 0s 40ms/step - loss: 31.6007 - mse: 3912.2817
Epoch 2800/3000
1/1 [==============================] - 0s 7ms/step - loss: 30.7847 - mse: 3757.8943
Epoch 2801/3000
1/1 [==============================] - 0s 9ms/step - loss: 30.8080 - mse: 3725.6880
Epoch 2802/3000
1/1 [==============================] - 0s 8ms/step - loss: 31.1563 - mse: 3770.5732
Epoch 2803/3000
1/1 [==============================] - 0s 9ms/step - loss: 31.5499 - mse: 3806.8738
Epoch 2804/3000
1/1 [==============================] - 0s 8ms/step - loss: 31.0364 - mse: 3765.9280
Epoch 2805/3000
1/1 [==============================] - 0s 45ms/step - loss: 31.2441 - mse: 3818.1956
Epoch 2806/3000
1/1 [==============================] - 0s 8ms/step - loss: 30.7770 - mse: 3699.2188
Epoch 2807/3000
1/1 [==============================] - 0s 8ms/step - loss: 30.9584 - mse: 3667.586

Epoch 2880/3000
1/1 [==============================] - 0s 8ms/step - loss: 31.0304 - mse: 3765.3521
Epoch 2881/3000
1/1 [==============================] - 0s 7ms/step - loss: 31.7099 - mse: 3834.3562
Epoch 2882/3000
1/1 [==============================] - 0s 6ms/step - loss: 31.3182 - mse: 3880.0195
Epoch 2883/3000
1/1 [==============================] - 0s 7ms/step - loss: 30.9719 - mse: 3763.0361
Epoch 2884/3000
1/1 [==============================] - 0s 7ms/step - loss: 31.1444 - mse: 3784.1768
Epoch 2885/3000
1/1 [==============================] - 0s 41ms/step - loss: 30.4817 - mse: 3728.7476
Epoch 2886/3000
1/1 [==============================] - 0s 9ms/step - loss: 30.6714 - mse: 3666.3594
Epoch 2887/3000
1/1 [==============================] - 0s 7ms/step - loss: 30.5459 - mse: 3735.2856
Epoch 2888/3000
1/1 [==============================] - 0s 7ms/step - loss: 31.2473 - mse: 3825.6802
Epoch 2889/3000
1/1 [==============================] - 0s 7ms/step - loss: 30.5933 - mse: 3721.1663

Epoch 2962/3000
1/1 [==============================] - 0s 8ms/step - loss: 30.5040 - mse: 3785.2439
Epoch 2963/3000
1/1 [==============================] - 0s 6ms/step - loss: 30.9636 - mse: 3743.6536
Epoch 2964/3000
1/1 [==============================] - 0s 47ms/step - loss: 30.8780 - mse: 3808.0188
Epoch 2965/3000
1/1 [==============================] - 0s 7ms/step - loss: 30.7229 - mse: 3711.1638
Epoch 2966/3000
1/1 [==============================] - 0s 7ms/step - loss: 30.8244 - mse: 3688.9888
Epoch 2967/3000
1/1 [==============================] - 0s 6ms/step - loss: 30.9924 - mse: 3657.7227
Epoch 2968/3000
1/1 [==============================] - 0s 9ms/step - loss: 30.5182 - mse: 3740.7844
Epoch 2969/3000
1/1 [==============================] - 0s 7ms/step - loss: 30.7681 - mse: 3756.5562
Epoch 2970/3000
1/1 [==============================] - 0s 42ms/step - loss: 30.9708 - mse: 3755.5789
Epoch 2971/3000
1/1 [==============================] - 0s 7ms/step - loss: 30.6919 - mse: 3773.053

In [36]:
predict_size = 60

pred_array = np.zeros(predict_size).reshape(1,predict_size,1)
X_test_initial = timeseries[1:].reshape(1,length,1)
print(X_test_initial)
pred_array = model.predict(X_test_initial)# if predictions of training samples required

#forecast is created by predicting next future value based on previous predictions
pred_array[:,0,:] = model.predict(X_test_initial)[:,-1:,:]
for i in range(predict_size-1):
    print(i)
    print(pred_array)
    pred_array[:,i+1:,:] = model.predict(np.append(X_test_initial[:,i+1:,:], 
                           pred_array[:,:i+1,:]).reshape(1,length,1))[:,-1:,:]


[[[  1]
  [  1]
  [  4]
  [  3]
  [  2]
  [  5]
  [  4]
  [  8]
  [  5]
  [ 18]
  [ 27]
  [ 28]
  [ 41]
  [ 49]
  [ 36]
  [133]
  [ 97]
  [168]
  [196]
  [439]
  [175]
  [368]
  [349]
  [345]
  [475]
  [427]
  [627]
  [793]
  [651]
  [601]
  [743]
  [683]
  [712]
  [919]
  [889]
  [756]
  [812]
  [837]
  [727]]]
0
[[[7.4432593e+02]
  [1.3536093e+00]
  [7.8590262e-01]
  [3.3665731e+00]
  [3.6650770e+00]
  [6.9823354e-01]
  [2.9402413e+00]
  [8.3829880e+00]
  [8.4767075e+00]
  [1.6180361e+01]
  [1.8065186e+01]
  [3.1127356e+01]
  [3.0887827e+01]
  [5.3993176e+01]
  [7.2179657e+01]
  [1.0944725e+02]
  [1.5777321e+02]
  [1.9486037e+02]
  [2.2819330e+02]
  [1.7757413e+02]
  [2.7927332e+02]
  [3.5185623e+02]
  [3.4557947e+02]
  [4.7987900e+02]
  [4.8332867e+02]
  [5.2316052e+02]
  [6.0962347e+02]
  [6.5345612e+02]
  [7.2726233e+02]
  [7.4465723e+02]
  [7.7714136e+02]
  [7.1383099e+02]
  [7.9000861e+02]
  [8.1697766e+02]
  [7.9732532e+02]
  [8.1453613e+02]
  [8.3537482e+02]
  [7.2555103e+02]


42
[[[744.3259  ]
  [713.39923 ]
  [636.57635 ]
  [599.98663 ]
  [508.96335 ]
  [441.2078  ]
  [382.38132 ]
  [325.92838 ]
  [278.4417  ]
  [241.83932 ]
  [193.71564 ]
  [142.05243 ]
  [110.47146 ]
  [ 74.14475 ]
  [ 37.355824]
  [ 17.387886]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]]]
43
[[[744.3259  ]
  [713.39923 ]
  [636.57635 ]
  [599.98663 ]
  [508.96335 ]
  [441.2078  ]
  [382.38132 ]
  [325.92838 ]
  [278.4417  ]
  [241.83932 ]
  [193.71564 ]
  [142.05243 ]
  [110.47146 ]
  [ 74.14475 ]
  [ 37.355824]
  [ 17.387886]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  [  0.      ]
  

In [37]:
ravd = pred_array.ravel()
print(ravd)
ravd.sum()

[744.3259   713.39923  636.57635  599.98663  508.96335  441.2078
 382.38132  325.92838  278.4417   241.83932  193.71564  142.05243
 110.47146   74.14475   37.355824  17.387886   0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.      ]


5448.1777